In [4]:
import torch; 
print(torch.__version__)
print(torch.version.cuda)

1.6.0
10.2


In [5]:
import torch; 
print(torch.version.cuda)

10.2


In [6]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html


In [7]:
import os.path as osp
import argparse

import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from tqdm import tqdm
from IPython.core.debugger import set_trace

In [3]:
!conda env list

# conda environments:
#
base                  *  /home/user/conda



In [8]:
def edge_pred_prep(data):
    E = data.edge_index.T.tolist()
    size = len(set([n for e in E for n in e])) 
    adjacency = [[0]*size for _ in range(size)]
    for sink, source in E:
        adjacency[sink][source] = 1

    yy = []
    for i in range(len(data.x)):
        for j in range(i, len(data.x)):
              yy.append(adjacency[i][j])

    y_edge = torch.Tensor(yy).int()
    data.y = y_edge.long()
    
    mask = [False for _ in range(len(y_edge))]
    train_mask = mask.copy()
    val_mask = mask.copy()
    test_mask = mask.copy()
    
    for i in range(int(len(y_edge)*1/100)):
        train_mask[i] = True

    for i in range(int(len(y_edge)*1/100), int(len(y_edge)*2/3)):
        val_mask[i] = True

    for i in range(int(len(y_edge)*1/100), len(y_edge)):
        test_mask[i] = True
        
    data.train_mask = torch.Tensor(train_mask).bool()
    data.val_mask = torch.Tensor(val_mask).bool()
    data.test_mask = torch.Tensor(test_mask).bool()

In [ ]:
SENTENSES_SIZE = 10

In [9]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(1433, 16, cached=True,
                             normalize=True)
        self.conv2 = GCNConv(16, 2, cached=True,
                             normalize=True)

    def forward(self):
        
        def doc_embedding(x):
            doc_embeddings = torch.empty([len(SENTENSES), 2], device=torch.device('cuda:0'))
            for i in tqdm(len(SENTENSES)):
                for j in range(SENTENSES[i]):
                    doc_embedding[i] += x[SENTENSES[i][j]]
            return doc_embeddings
                
        
        def edge_embedding(x):
            index = 0
            edge_embeddings = torch.empty([3667986, 2], device=torch.device('cuda:0'))
            print(len(x))
            for i in tqdm(range(len(x))):
                for j in range(i, len(x)):
                    
                    edge_embeddings[index] = torch.mul(x[i], x[j])
                    index += 1
            return edge_embeddings

        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, None))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, None)
        print(len(x))
        print(len(x[0]))
        edge_x = edge_embedding(x)
        return F.log_softmax(edge_x, dim=1)

def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()
    return model()


@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
#       print(logits[mask].max(1))
#        print(mask)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return [accs, pred]

In [ ]:
edge_index = torch.tensor(np.array(lst).T, dtype=torch.long)

In [ ]:
train_mask = []
test_mask = []
for i in range(len(graph)):
    train_mask.append(True)
    test_mask.append(False)
for i in range(int(len(graph)*4/5), len(graph)):
    train_mask[i] = False
    test_mask[i] = True

In [ ]:
train_mask = torch.tensor(train_mask, dtype=torch.bool)
test_mask = torch.tensor(test_mask, dtype=torch.bool)

In [ ]:
x = torch.tensor(graph_enc.drop(["edge_id", "adjacent"], axis = 1).to_numpy(), dtype=torch.float)
y = torch.tensor(graph_enc["length"].to_numpy(), dtype=torch.float)
data = Data(x=x, edge_index=edge_index, train_mask = train_mask, test_mask = test_mask)

In [10]:
from torch_geometric.datasets import KarateClub
from torch_geometric.datasets import Planetoid

In [11]:
#dataset = KarateClub()
#data = dataset[0]

In [12]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

Processing...
Done!


In [13]:
edge_pred_prep(data)

In [14]:
data

Data(edge_index=[2, 10556], test_mask=[3667986], train_mask=[3667986], val_mask=[3667986], x=[2708, 1433], y=[3667986])

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)  # Only perform weight-decay on first convolution.


In [ ]:
dictionary = ['cake', "is", "a", 'lie', 'or', 'fake']
senteses = ['Cake is a lie', 'Is cake a lie or a fake?']
WINDOW_SIZE = 3

In [ ]:
def PMI(i, j):
    
def p(i, j):
    
    
def p(i):
    

In [ ]:
for i in range(len(dictionary)):
    for j in range(i, len(dictionary)):
        
            

In [16]:
output = 0
best_val_acc = test_acc = 0
for epoch in range(1, 201):
    A = train()
    train_acc, val_acc, tmp_test_acc = test()[0]
    output = test()[1]
#    if val_acc > best_val_acc:
#        best_val_acc = val_acc
    test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_acc, test_acc))

  0%|          | 2/2708 [00:00<04:04, 11.05it/s]

2708
2
2708


  0%|          | 2/2708 [00:00<03:51, 11.67it/s]

2708
2
2708


  0%|          | 2/2708 [00:00<02:42, 16.68it/s]

...

2708
2
2708


  0%|          | 2/2708 [00:00<04:09, 10.86it/s]

Epoch: 002, Train: 0.9990, Test: 0.9986
2708
2
2708


100%|██████████| 2708/2708 [02:02<00:00, 22.16it/s] 


KeyboardInterrupt: 

In [ ]:
epoch 

In [ ]:
%debug

In [15]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
